In [ ]:
import itertools
import numpy as np
import os
from tqdm import tqdm
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd


from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
import functools

%load_ext autoreload
%autoreload 2

from genpen.utils import Paper
import bezier

In [ ]:
# make page
paper_size = '17x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
def random_line_subdivide(poly, x0=None, x1=None):
    if x0 is None:
        x0 = np.random.uniform(0,1)
    if x1 is None:
        x1 = (x0 + 0.5) % 1
    return LineString([poly.boundary.interpolate(x, normalized=True) for x in [x0, x1]])

In [ ]:
def random_bezier_subdivide(poly, x0=None, x1=None, n_eval_points=50):
    if x0 is None:
        x0 = np.random.uniform(0.2, 0.4)
    if x1 is None:
        x1 = np.random.uniform(0.6, 0.8)
    line = np.asfortranarray(random_line_subdivide(poly, x0, x1))
    bez_array = np.stack([line[0], poly.centroid, line[1]]).T
    curve1 = bezier.Curve(bez_array, degree=2)
    bez = curve1.evaluate_multi(np.linspace(0., 1., n_eval_points))
    return sg.asLineString(bez.T)

In [ ]:
def split_along_longest_side_of_min_rectangle(poly, xgen=None):
    if xgen is None:
        xgen = gp.make_callable(0.5)
    mrrc = poly.minimum_rotated_rectangle.boundary.coords
    sides = [LineString([mrrc[i], mrrc[i+1]]) for i in range(4)]
    longest_sides = [sides[i] for i in np.argsort([-l.length for l in sides])[:2]]
    bps = [ls.interpolate(xgen(), normalized=True)for ls in longest_sides]

    return LineString([so.nearest_points(bp, poly.boundary)[1] for bp in bps])

In [ ]:
def recursive_split(poly, split_func=random_line_subdivide, p_continue=0.7, depth=0, depth_limit=15, buffer_kwargs=None):
    
    if buffer_kwargs is None:
        buffer_kwargs = {'distance':0}
    polys = list(poly.difference(split_func(poly).buffer(1e-6)))
    split_polys = []
    
    for i, p in enumerate(polys):
        continue_draw = np.random.binomial(n=1, p=p_continue)
        
        if continue_draw and (depth<depth_limit):
            
            split_polys += recursive_split(
                p, split_func=split_func, p_continue=p_continue, 
                depth=depth+1, depth_limit=depth_limit,
                buffer_kwargs=buffer_kwargs
            ) 
        else:
            split_polys.append(p.buffer(**buffer_kwargs))
    return split_polys

In [ ]:
def recursive_split_frac_buffer(poly, split_func=random_line_subdivide, p_continue=0.7, depth=0, depth_limit=15, buffer_kwargs=None, buffer_frac=-0.1):
    try:
        if buffer_kwargs is None:
            buffer_kwargs = {'join_style':2, 'cap_style':2}
        polys = list(poly.difference(split_func(poly).buffer(1e-6)))
        split_polys = []

        for i, p in enumerate(polys):
            continue_draw = np.random.binomial(n=1, p=p_continue)
            distance=p.centroid.distance(p.boundary)*buffer_frac
            bp = p.buffer(distance=distance, **buffer_kwargs)
            if continue_draw and (depth<depth_limit):

                split_polys += recursive_split_frac_buffer(
                    bp, split_func=split_func, p_continue=p_continue, 
                    depth=depth+1, depth_limit=depth_limit,
                    buffer_kwargs=buffer_kwargs, buffer_frac=buffer_frac
                ) 
            else:

                split_polys.append(bp)
        return split_polys
    except:
        return [poly]

In [ ]:
xgen = ss.uniform(loc=0.4, scale=0.02).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.93, 
    depth=0, 
    depth_limit=12,
    buffer_frac=-0.01
)

bps = MultiPolygon([p for p in splits])

nbps = []
for p in bps:
    xbounds = drawbox.bounds[0], drawbox.bounds[2]
    d = np.interp(p.centroid.x, xbounds,  [-0.1, -1.4],)
    nbps.append(p.buffer(d, cap_style=2, join_style=2))
bps = MultiPolygon(nbps)    

sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/207_subdivide.svg')

In [ ]:
split_func = functools.partial(random_bezier_subdivide, x0=0.1, x1=0.7, n_eval_points=20)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func, 
    p_continue=0.96, 
    depth=0, 
    depth_limit=12,
    buffer_frac=-0.02
)


bps = MultiPolygon([p for p in splits])

nbps = []
for p in bps:
    xbounds = drawbox.bounds[0], drawbox.bounds[2]
    d = np.interp(p.centroid.x, xbounds,  [-0.1, -1.4],)
    nbps.append(p.buffer(d, cap_style=2, join_style=2))
bps = MultiPolygon(nbps)    
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/208_subdivide.svg')

In [ ]:
split_func = functools.partial(random_bezier_subdivide, x0=0.3, x1=0.7, n_eval_points=20)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func, 
    p_continue=0.99, 
    depth=0, 
    depth_limit=13,
    buffer_frac=-0.01
)

bps = MultiPolygon([p for p in splits])
nbps = []
for p in bps:
    xbounds = drawbox.bounds[0], drawbox.bounds[2]
    d = np.interp(p.centroid.x, xbounds,  [-0.1, -0.6],)
    nbps.append(p.buffer(d, cap_style=2, join_style=2))
bps = MultiPolygon(nbps)   

sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/210_subdivide.svg')

# depth dependent
getting pretty dumb to keep repeating this, hard to handle namespace tho it should probs be a class

In [ ]:
def recursive_split_frac_buffer_depth(poly, p_continue=0.7, depth=0, depth_limit=15, buffer_kwargs=None, buffer_frac=-0.1):
    try:
        if buffer_kwargs is None:
            buffer_kwargs = {'join_style':2, 'cap_style':2}
        xgen = ss.uniform(loc=0.2 + 0.00*depth, scale=0.8+0.00*depth).rvs
        split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
        polys = list(poly.difference(split_func(poly).buffer(1e-6)))
        split_polys = []

        for i, p in enumerate(polys):
            continue_draw = np.random.binomial(n=1, p=p_continue)
            distance=p.centroid.distance(p.boundary)*buffer_frac
            bp = p.buffer(distance=distance, **buffer_kwargs)
            if continue_draw and (depth<depth_limit):

                split_polys += recursive_split_frac_buffer(
                    bp, split_func=split_func, p_continue=p_continue, 
                    depth=depth+1, depth_limit=depth_limit,
                    buffer_kwargs=buffer_kwargs, buffer_frac=buffer_frac
                ) 
            else:

                split_polys.append(bp)
        return split_polys
    except:
        return [poly]

In [ ]:
xgen = ss.uniform(loc=0.45, scale=0.3).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.93, 
    depth=0, 
    depth_limit=12,
    buffer_frac=-0.02
)


In [ ]:
bps = MultiPolygon([p for p in splits])

In [ ]:
stp = gp.ScaleTransPrms(n_iters = 20, d_buffer=-0.15, d_translate_factor=0.2)

big_bps = [gp.Poly(p) for p in bps if p.area>100]

for p in big_bps:
    p.fill_scale_trans(**stp.prms)

fills = gp.merge_LineStrings([p.fill for p in big_bps])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.stroke(1)
sk.geometry(bps.boundary)
sk.stroke(2)
sk.geometry(fills)

sk.display(color_mode='none')

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/204_subdivide.svg')

In [ ]:
xgen = ss.uniform(loc=0.4, scale=0.4).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.97, 
    depth=0, 
    depth_limit=12,
    buffer_frac=-0.0
)


In [ ]:
bps = MultiPolygon([p for p in splits])

nbps = []
for p in bps:
    xbounds = drawbox.bounds[0], drawbox.bounds[2]
    d = np.interp(p.centroid.x, xbounds,  [-0.1, -1.1],)
    nbps.append(p.buffer(d, cap_style=2, join_style=2))
bps = MultiPolygon(nbps)    

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.stroke(1)
sk.geometry(bps.boundary)
sk.stroke(2)
# sk.geometry(fills)

sk.display(color_mode='none')

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/205_subdivide.svg')

In [ ]:
xgen = ss.uniform(loc=0.5, scale=0.3).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.96, 
    depth=0, 
    depth_limit=13,
    buffer_frac=-0.01
)


In [ ]:
bps = MultiPolygon([p for p in splits])

nbps = []
for p in bps:
    xbounds = drawbox.bounds[0], drawbox.bounds[2]
    d = np.interp(p.centroid.x, xbounds,  [-0.15, -1.6],)
    nbps.append(p.buffer(d, cap_style=2, join_style=2))
bps = MultiPolygon(nbps)    

In [ ]:


fbps = []
for _p in bps:
    area = np.log10(_p.area)
    p = gp.Poly(_p)
    n_iters = int(np.interp(area, [-3, 4.3], [1, 230]))
    stp = gp.ScaleTransPrms(n_iters = n_iters, d_buffer=-0.25, d_translate_factor=0.7, angles=np.random.uniform(0,np.pi*2))
    db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=4.5, step_mu=0.)
    db_mults[db_mults<0.] = 0.05
    stp.d_buffers *= db_mults
    stp.angles = np.linspace(0, np.radians(360*0.7), len(stp.d_buffers))
    p.fill_scale_trans(**stp.prms)
    fbps.append(p)

fills = gp.merge_LineStrings([p.fill for p in fbps])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.stroke(1)
sk.geometry(bps.boundary)
sk.stroke(2)
sk.geometry(fills)

sk.display(color_mode='none')

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesimplify --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/212_subdivide.svg')

In [ ]:
np.

In [ ]:

split_func = functools.partial(random_bezier_subdivide, x0=0.3, x1=0.7, n_eval_points=20)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func, 
    p_continue=0.99, 
    depth=0, 
    depth_limit=13,
    buffer_frac=-0.01
)



In [ ]:
bps = MultiPolygon([p for p in splits])

nbps = []
for p in bps:
    xbounds = drawbox.bounds[0], drawbox.bounds[2]
    d = np.interp(p.centroid.x, xbounds,  [-0.2, -0.45],)
    nbps.append(p.buffer(d, cap_style=2, join_style=2))
bps = MultiPolygon(nbps)    

In [ ]:
3 + 4 - 10

In [ ]:
3.1 + 4.1 - 9.9

In [ ]:
3.2 + 4.2 - 9.8

In [ ]:


fbps = []
for _p in bps:
    area = np.log10(_p.area)
    p = gp.Poly(_p)
    n_iters = int(np.interp(area, [-3, 4.3], [1, 230]))
    stp = gp.ScaleTransPrms(n_iters = n_iters, d_buffer=-0.25, d_translate_factor=0.7, angles=np.random.uniform(0,np.pi*2))
    db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=4.5, step_mu=0.)
    db_mults[db_mults<0.] = 0.05
    stp.d_buffers *= db_mults
    stp.angles = np.linspace(0, np.radians(360*0.7), len(stp.d_buffers))
    p.fill_scale_trans(**stp.prms)
    fbps.append(p)

fills = gp.merge_LineStrings([p.fill for p in fbps])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.stroke(1)
sk.geometry(bps.boundary)
sk.stroke(2)
sk.geometry(fills)

sk.display(color_mode='none')

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesimplify --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/226_subdivide.svg')